Run some attacks on DP-CGANS

In [36]:
import pandas as pd 
import os 
import tapas.datasets
import tapas.generators
import json I do not unde


In [37]:
datapath = "../datasets"
dataset_name = "Adult"
model_name = "DPCGANS"
file = f"Real/real_{dataset_name.lower()}_data.csv"

In [38]:
df = pd.read_csv(os.path.join(datapath, dataset_name, file), index_col=0)

In [39]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,27,Private,177119,Some-college,10,Divorced,Adm-clerical,Unmarried,White,Female,0,0,44,United-States,<=50K
1,27,Private,216481,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,<=50K
2,25,Private,256263,Assoc-acdm,12,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K
3,46,Private,147640,5th-6th,3,Married-civ-spouse,Transport-moving,Husband,Amer-Indian-Eskimo,Male,0,1902,40,United-States,<=50K
4,45,Private,172822,11th,7,Divorced,Transport-moving,Not-in-family,White,Male,0,2824,76,United-States,>50K


In [40]:
var = "fnlwgt"
df[var].unique()

array([177119, 216481, 256263, ...,  45317, 215862, 186925])

I created a json file with the data schema in `data_schemas/adult.json`. I am not sure about all the "countable" data types, but none of the input data seems continuous/to have decimals. See the tapas documentation: https://privacy-sdg-toolbox.readthedocs.io/en/latest/dataset-schema.html

In [41]:
with open('data_schemas/adult.json') as file:
    # Load the JSON data into a dictionary
    data_schema = json.load(file)


In [42]:
data_description = tapas.datasets.DataDescription(schema=data_schema)
data = tapas.datasets.TabularDataset(data=df, description=data_description)

In [43]:
generator = tapas.generators.GeneratorFromExecutable(exe="src/generator_from_dataset.py")
generator.fit(data)

In [44]:
synthetic_data = generator.generate(2000)

### Run an attack

import tapas.datasets
import tapas.generators
import tapas.threat_models
import tapas.attacks
import tapas.report

In [45]:
import tapas.datasets
import tapas.generators
import tapas.threat_models
import tapas.attacks
import tapas.report

In [46]:
data_knowledge = tapas.threat_models.AuxiliaryDataKnowledge(
      data,
      auxiliary_split=0.5,
      num_training_records=5000
)
sdg_knowledge = tapas.threat_models.BlackBoxKnowledge(
    generator,
    num_synthetic_records=5000,
)

In [47]:
threat_model = tapas.threat_models.TargetedMIA(
   attacker_knowledge_data=data_knowledge,
   target_record=data.get_records(range(50)),
   attacker_knowledge_generator=sdg_knowledge,
   generate_pairs=True,
   replace_target=True
)

### Closest distance MIA

In [48]:
attacker = tapas.attacks.ClosestDistanceMIA()
attacker.train(threat_model, num_samples=10)
attack_summary = threat_model.test(attacker, num_samples=100)
metrics = attack_summary.get_metrics()


/home/flavio/repositories/projects/GANS/explore-attacks/.venv/lib/python3.9/site-packages/tapas/report/attack_summary.py:250: RuntimeWarning: invalid value encountered in divide
  return np.log(max(np.max(tp/fp), np.max((1-fp)/(1-tp))))


In [49]:
metrics.head()

,dataset,target_id,generator,attack,accuracy,true_positive_rate,false_positive_rate,mia_advantage,privacy_gain,auc,effective_epsilon
0,Unnamed dataset (AUX),0,src/generator_from_dataset.py,"ClosestDistance(Hamming, accuracy)",0.5,0.0,0.0,0.0,1.0,0.5,0.0


### Local neighborhood attack

It is important to re-define the threat model; otherwise I think it takes the existing trained model from above.

In [51]:
threat_model = tapas.threat_models.TargetedMIA(
   attacker_knowledge_data=data_knowledge,
   target_record=data.get_records(range(50)),
   attacker_knowledge_generator=sdg_knowledge,
   generate_pairs=True,
   replace_target=True
)

In [52]:
attacker = tapas.attacks.LocalNeighbourhoodAttack()
attacker.train(threat_model, num_samples=10)
attack_summary = threat_model.test(attacker, num_samples=100)
metrics = attack_summary.get_metrics()

/home/flavio/repositories/projects/GANS/explore-attacks/.venv/lib/python3.9/site-packages/tapas/report/attack_summary.py:250: RuntimeWarning: invalid value encountered in divide
  return np.log(max(np.max(tp/fp), np.max((1-fp)/(1-tp))))


In [53]:
metrics.head()

,dataset,target_id,generator,attack,accuracy,true_positive_rate,false_positive_rate,mia_advantage,privacy_gain,auc,effective_epsilon
0,Unnamed dataset (AUX),0,src/generator_from_dataset.py,"LocalNeighbourhood(Hamming, 1, accuracy)",0.5,0.0,0.0,0.0,1.0,0.5,0.0


It works mechanically, but it is unclear what is going on:
- one row per data set 
- TPR, FPR are 0. why?
- does it matter whether the generated data are deterministic when reading from a file? 
- difficult to understand the outputs above. need a better understanding
    - of the attacks: how do they work, what do they need, what are their assumptions
    - of the library: understand in more details how the library implements attacks
